In [ ]:
shift_param = 1

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
import os
import datetime as dt
import pickle

from src.utils import fetch_latest_fpl_data

from sklearn.linear_model import LogisticRegression
from scipy.stats import poisson

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

In [ ]:
# model for estimating bonus points based on gameweek bps
model_path = Path(f"../models/logistic_regression_for_bonus_points.pkl")
with open(model_path, "rb") as f:
    clf = pickle.load(f)

# Functions

In [ ]:
def fpl_data_processing(df, columns):

    xg_data = []
    xa_data = []
    xga_data = []
    for ix, row in df.iterrows():
        my_gameweek = row['gameweek']
        xg_data.append( row[f'xG_week{my_gameweek}'] )
        xa_data.append( row[f'xA_week{my_gameweek}'] )
        xga_data.append( row[f'xGA_week{my_gameweek}'] )

    df['gameweek_xG'] = xg_data
    df['gameweek_xA'] = xa_data
    df['gameweek_xGA'] = xga_data

    df_new = df[columns].copy()

    return df_new

In [ ]:
def my_fill_na(x, gameweek_col, diff_col):
    '''Fill nan values for first items for grouped variables where diff is calculated. But also don't fill for season 22-23,
    where data is missing for a number of weeks at the beginning of the season.'''
    my_value = x[diff_col] if (np.isnan(x[gameweek_col])) & (x['minutes']<=90) else x[gameweek_col]
    return my_value
    

In [ ]:
def calculate_xPoints(x,clf):
    """Expected points for a given gameweek given underlying stats for that gameweek."""

    clean_sheet_points = np.array([4,4,1,0])
    goal_points = np.array([6,6,5,4])

    # calculate expexted points
    points_played = np.array([1 if x['gameweek_minutes']>0 else 0])
    points_played_over_60 = np.array([1 if x['gameweek_minutes']>=60 else 0])
    points_xG = goal_points[x['element_type']-1] * x['gameweek_xG']
    points_xA = x['gameweek_xA'] * 3
    clean_sheet_probability = np.array(poisson.pmf(0,x['team_xGA']))
    points_clean_sheet = [clean_sheet_points[x['element_type']-1] * clean_sheet_probability if x['gameweek_minutes']>=60 else 0]
    points_saves = x['gameweek_saves'] // 3
    points_penalty_saves = x['gameweek_penalties_saved'] * 5 * 0.21 #points for save times approx. probability of penalty save
    #penalty_for_penalty_miss = x['Performance_PKatt'] * (-2*0.21) # this data only on fbref
    # estimate bonus points
    if not np.isnan(x['gameweek_bps']):
        y_pred_prob = clf.predict_proba(np.array(x['gameweek_bps']).reshape(-1, 1))
    else:
        # return nan if bonus points can't be estimated 
        return np.nan
    points_bonus = np.matmul(y_pred_prob, np.array([0,1,2,3]).reshape((4,1)))
    
    # penalty for possible points deductions based on goals conceded
    xGA = x['team_xGA']
    # calculate penalty
    xGA_conceded_penalty = -(poisson.pmf(2,xGA)+poisson.pmf(3,xGA))-(poisson.pmf(4,xGA)+poisson.pmf(5,xGA))-(poisson.pmf(6,xGA)+poisson.pmf(7,xGA))-(poisson.pmf(8,xGA)+poisson.pmf(9,xGA)-(poisson.pmf(10,xGA)+poisson.pmf(11,xGA)))
    # apply penalty only to GK and DEF
    if (x['element_type']==1) | (x['element_type']==2):
        xGA_conceded_penalty = xGA_conceded_penalty
    else:
        xGA_conceded_penalty = 0
    # scale penalty with playing time
    xGA_conceded_penalty = (x['gameweek_minutes'] / 90) * xGA_conceded_penalty

    penalty_for_cards = [-3 if x['gameweek_red_cards']==1 else -1 if x['gameweek_yellow_cards']==1 else 0]
    penalty_for_own_goal = -2 * x['gameweek_own_goals']

    # add up all point components
    total_points = float(points_played + points_played_over_60 + points_xG + points_xA + points_clean_sheet + points_saves +\
                    points_penalty_saves + points_bonus + xGA_conceded_penalty +\
                    penalty_for_cards + penalty_for_own_goal)
    
    return total_points

# Fetch data

In [ ]:
# fpl data from previous seasons
filepath = Path('../data/modeling/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0)
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
# fpl data from this season
fpl_df_new = fetch_latest_fpl_data()
display(fpl_df_new.head())
display(fpl_df_new.shape)

In [ ]:
# concatenate new fpl data with old
fpl_df = pd.concat([fpl_df, fpl_df_new], join='outer').reset_index(drop=True)
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
# running team data from past seasons
filepath = Path('../data/modeling/team_data.csv')
team_data = pd.read_csv(filepath, index_col=0)
display(team_data.head())
display(team_data.tail())
display(team_data.shape)

In [ ]:
# fpl fixtures data from this season
filepath = Path('../data/fixtures/fpl_fixtures.csv')
fixtures_fpl = pd.read_csv(filepath, index_col=0)
fixtures_fpl = fixtures_fpl[fixtures_fpl.finished]
display(fixtures_fpl.head())
display(fixtures_fpl.shape)

In [ ]:
# fbref fixtures data from this season
filepath = Path('../data/fixtures/fbref_fixtures.csv')
fixtures_fbref = pd.read_csv(filepath, index_col=0)
display(fixtures_fbref.head())
display(fixtures_fbref.shape)

# Process FPL data

In [ ]:
# find how many minutes a player played on a given gameweek
fpl_df['gameweek_minutes'] = fpl_df.groupby(['first_name', 'second_name', 'season'])['minutes'].diff()
# fill na caused at the start of each season by taking diff (but don't fill for season 22-23 where early season data is missing)
fpl_df['gameweek_minutes'] = fpl_df.apply(lambda x: my_fill_na(x, 'gameweek_minutes', 'minutes'), axis=1)
print('Number of rows with zero minutes played in a gameweek:')
display(fpl_df[fpl_df.gameweek_minutes==0].shape[0])
print('Number of rows with over 90 minutes played in a gameweek:')
display(fpl_df[fpl_df.gameweek_minutes>90].shape[0])

In [ ]:
# check does the latest season have any problem data (ok if '23-24' does not appear here)
display(fpl_df.loc[fpl_df.gameweek_minutes>90, 'season'].unique())
display(fpl_df.loc[fpl_df.gameweek_minutes==0, 'season'].unique())

In [ ]:
# drop rows with 0 minutes or more than 90 minutes
fpl_df = fpl_df[(fpl_df.gameweek_minutes>0) & (fpl_df.gameweek_minutes<=90)].reset_index(drop=True)
display(fpl_df.head())
display(fpl_df.shape)

### Add xG data to FPL fixtures data

In [ ]:
# map fbref team names to fpl team names
fbref_teams = np.sort(pd.concat([fixtures_fbref.Home, fixtures_fbref.Away]).unique())
team_name_dict = dict(zip(fbref_teams, np.sort(fixtures_fpl.home_team.unique())))
display(team_name_dict)

fixtures_fbref['Home'] = fixtures_fbref['Home'].apply(lambda x: team_name_dict[x])
fixtures_fbref['Away'] = fixtures_fbref['Away'].apply(lambda x: team_name_dict[x])
display(fixtures_fbref.head())

In [ ]:
home_xg = []
away_xg = []
for ix, row in fixtures_fpl.iterrows():
    home_team = row.home_team
    away_team = row.away_team
    home_team_xg = fixtures_fbref.loc[(fixtures_fbref['Home']==home_team) & (fixtures_fbref['Away']==away_team), 'xG_home'].values[0]
    away_team_xg = fixtures_fbref.loc[(fixtures_fbref['Home']==home_team) & (fixtures_fbref['Away']==away_team), 'xG_away'].values[0]
    home_xg.append( home_team_xg )
    away_xg.append( away_team_xg )

fixtures_fpl['xg_home'] = home_xg
fixtures_fpl['xg_away'] = away_xg

display(fixtures_fpl.head())
print('Nulls:')
display(fixtures_fpl[['xg_home', 'xg_away']].isnull().sum())